## Tree
### typically Binary Tree
<b>Binary Tree</b>:   
a tree data structure in which each node has at most two children, which are referred to as the left child and the right child  
<b>Binary Search Tree</b>:    
also called an ordered or sorted binary tree, is a rooted binary tree whose internal nodes each store a key greater than all the keys in the node's left subtree and less than those in its right subtree.

<b>Tree Traversal</b>:   
<b>inorder</b>: just suitable for binary tree, not suitable to be extended to N-ary tree  
traverse the left subtree => visit the root node => traverse the right subtree  
root is in the middle => in  
<b>preorder</b>:  
visit the root node => traverse the left subtree => traverse the right subtree  
root is visited first => pre  
<b>postorder</b>:  
traverse the left subtree => traverse the right subtree => visit the root node  
root is visted last => post  
<b>level-order</b>:  
traverse the tree level by level  

<b>199.Binary Tree Right Side View </b>  
Keep track of level using the Len of list. So each level only add the right most value.    
If Len(list) == level, meaning loop into a new level, and can add a new value, which is the rightest value  
Traversal order is root, right, left, to make sure the rightest value of each level is first added.  

In [1]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right

In [2]:
def rightSideView(root):
    """DFS way"""
    def helper(node, result, level):
        if not node:
            return
        if len(result) == level:  # meaning entering into a new level of the tree
            result.append(node.val)
        helper(node.right, result, level+1)   # right first to ensure meet the requirement of uestion
        helper(node.left, result, level+1)
    result = []
    helper(root, result, 0)
    
    return result

BFS way consumes more space than DFS way above. both O(N) for visiting all the nodes exactly once

In [1]:
from collections import deque
def rightSideView(root):
    """BFS way, pre-order traversal"""
    if not root:
        return []
    result = []
    queue = deque([root])
    while queue:
        size = len(queue)
        for _ in range(size): # for loop to ensure pop nodes from the same level (current level) => interesting way
            node = queue.popleft()
            val = node.val  # to store the last value in each level, which is the target val
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
        result.append(val)
    return result

<b>108.Convert Sorted Array to Binary Search Tree </b>  
still, inorder traversal: left, root, right, in this way the value is in ascending order.   
So first get the middle item of the ascending list, make it the root node. all the items in middle item’s left will be in the left tree part, all the items in the middle items’s right will be in the right tree part.  
<b>recursive</b>: 
<b>base case</b>: if array is empty, not valid, get none, otherwise keep getting middle item as root, and then make left tree, right tree  
Time complexity: O(N) since we visit each node exactly once.
Space complexity is the stack used for recursion, which for balanced tree should be O(logN).

In [3]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
def sortedArrayToBST(nums):
    if not nums:
        return None
    mid = len(nums)//2   # suppose 5 element, mid is 2, which is the third element, suppose 2 element, mid is 1, which is the second element
    node = TreeNode(nums[mid])
    node.left = sortedArrayToBST(nums[:mid])
    node.right = sortedArrayToBST(nums[mid+1:])
    return node

<b> 96 unique binary search trees </b>   
turns out to be a math and dynamic programming questions.
G(n): given n, how many structurally unqiue BST can be created  
F(i, n): having n node, use node i as the root, how many structurally unique BST can be created.  
F(i, n ) = G(i-1) * G(n-i) => using i as the root, there are i-1 nodes smaller than i, and n-i nodes greater than i. so F(i, n) is same to how many unique BST for using i-1 nodes and n-i nodes.   
easily we can get: G(n) is using each of the node as root, sum up all the unique BST number.  
G(N)=F(1,n)+F(2,n)+F(3,n) + .... + F(n,n)  
     = G(1-1)G(n-1) + G(2-1)G(n-2) + G(3-1)G(n-3)+...+G(n-1)G(0)   
i's range is [1, n] included, since i represents the node used as root.

for the base cases, G(0) = 1 (no nodes in the side), G(1)=1 (only one node in the side)
    

In [3]:
def numTrees(n):
    G = [0]*(n+1) # dp space for G 0 to n, G[n] is the result we want
    G[0] = 1
    G[1] = 1
    if n > 1:
        # node_num represents different possible n
        for node_num in range(2, n+1): 
            # i stands for the node used as root, value in the range of [1, node_num]
            for i in range(1, node_num+1):
                G[node_num] += G[i-1]*G[node_num-i]
    return G[n]

In [4]:
numTrees(3)

5

<b>95.Unique Binary Search Tree II </b>

way 1 is using recursive with binary search tree’s features  
binary search tree feature: [1,n], if i is root, then its left part is composed of [1,i-1], right part is composed of [i+1, n]  
recursive feature, now [1, i-1], [i+1, n] are the same Q as original  


In [4]:
def generateTrees(n):
    """
    BST feature: [1,n], if i is root, then its left part is composed of [1,i-1], right part is composed of [i+1, n]
    DFS, recursive, now [1, i-1], [i+1, n] are the same Q as original
    """
    def helper(n_list):
        if not n_list:
            return [None]  # why []? => result is a list
        result = []
        for i in range(len(n_list)):
            for left in helper(n_list[:i]):
                for right in helper(n_list[i+1:]):
                    node = TreeNode(n_list[i])
                    node.left = left
                    node.right = right
                    result += [node]
        return result
    
    if n == 0:
        return []
    return helper([i for i in range(1, n+1)])
            

way 2 is recursive with binary search tree’s features plus dynamic programming

In [5]:
def generateTrees(n):
    """dynamic programming way!!"""
    result = [[]*(n+1)]  # save space, length from 0 to n
    if n ==0:
        return result[0]
    
    result[0].append(None)   # None is a valid scenario for cases with 0 element
    for length in range(1, n+1):  # range [1, n] how many lements in the tree
        for i in range(length):  # range[0, length-1], can see i as number of nodes in the left tree
            for left_tree in result[i]:  # left tree can have 0 to length-1 element
                for right_tree in result[length-1-i]:   # total number of element is length, -1 for the root, -i for the left tree, leftover is the number of elements in right tree
                    node = TreeNode(i+1)
                    node.left = left_tree
                    node.right = clone(right_tree, i+1)   # i+1 is the offset, result only save the structure
                    result[length].append(node)
    return result[n]

def clone(node, offset):
    if not node:
        return None
    new_node = TreeNode(node.val+offset)
    new_node.left = clone(node.left, offset)
    new_node.right = clone(node.right, offset)
    return new_node

<b>501.Find Mode in Binary Search Tree </b>  
For BST, if travel inorder, prev_node <= curr_node<=next_node. Compare curr node with prev node, if they match, increase curr_count (find a duplicate), if they dont match, reset curr_count to 1.  
have a global max_count var. compare max_count with curr_count in each traversal. If two counts match, that means find a result with same count, append value to result list. if curr_count > max_count, update max_count, reset result list to [new_node_val]   
remember to initiate the prev_node_val to None and update the prev_node_val with current node value in each traversal

In [7]:
def findMode(root):
    """
    utilizing Binary search tree feature
    if inorder, prev<=curr<=next node's value
    """
    prev=None
    curr_count = 0
    max_count = 0
    result = []
    
    def helper(node):
        if not node:
            return 
        helper(node.left)   # inorder traversal
        
        if node.val!=prev:
            curr_count=1   # A NEW START
        else:
            curr_count+=1
        if curr_count == max_count:
            result.append(node.val)  # multiple mode
        elif curr_count > max_count:
            result = [node.val]   # reset result list
            max_count = curr_count
        prev = node.val
        
        helper(node.right)
        
    helper(root)
    return result
            

<b>543.Diameter of Binary Tree </b>  
for each node: 
    get the longest path of the given node: get max_left, max_right, then max (history_max, max_left+max_right), update the history_max  
    get the longest single children path: get max_left, max_right,  max(max_left, max_right) + 1  
start point: if node is null, return 0

the helper function does two things  
1.calculate the longest child path of a given node  
2.updating the outside class variable (the target var) while traversal.  
doing post order traversal, from bottom to top

In [8]:
def diameterOfBinaryTree(root):
    """
    DFS for everynode, calculate what its longest left and right children path
    post order traversal
    """
    max_legnth = 0
    def helper(node):
        # return max child length
        if not node:
            return 0
        # post order traversal, from bottom to top
        left = helper(node.left)
        right = helper(node.right)
        
        # to get longest diameter path per given node
        max_legnth = max(max_length, left+right)
        
        # to get longest child path
        return max(left, right)+1   # +1 for the node itself
    
    helper(root)
    return max_length

<b> 124. binary tree maximum path sum </b>

similar to problem <b>543</b>!

one def helper function does two things: one is to update the global var of max path sum, another is to return the current max child branch value.

In [8]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
class Solution:
    def __init__(self):
        self.max_value = float('-inf')
    def maxPathSum(self, root):
        
        def helper(node):
            """do two things"""
            if not node:
                return 0
            left = max(0, helper(node.left))  # if left branch's max value is negative, no need to add it
            right = max(0, helper(node.right))
            self.max_value = max(self.max_value, left+right+node.val)
            return max(left, right)+node.val # which child path has largest value

        helper(root)
        return self.max_value
    

In [9]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
a = Solution()
a.maxPathSum(root)

6

<b> 236. Lowest Common Ancestor of a binary tree </b>

so smart!  
If the current (sub)tree contains both p and q, then the function result is their LCA. If only one of them is in that subtree, then the result is that one of them. If neither are in that subtree, the result is None.

post order traversal, from bottom up. first result is the answer so can ensure it gets the lowest common ancestor

In [10]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
def lowestCommonAncestor(root, p, q):
    # LCA can be the root node itself
    if root in (None,p, q):
        return root
    left = lowestCommonAncestor(root.left, p, q)
    right = lowestCommonAncestor(root.right, p, q)
    # left and right are all none empty, they must be p and q, the LCA found
    if left and right:
        return root
    # either p, q, or None
    else:
        return left or right

<b>102.Binary Tree Level Order Traversal </b>  

key point is how to keep track of the level  
way 1: len(result) == level, then you are entering a new level  
way 2: two queues, one for current level, another keeps track of next level of nodes

In [6]:
def levelOrder(root):
    """DFS, using level and len of result to control where to add
        pre-order traversal
    """
    def helper(node, level):
        if not node:
            return
        if len(result) == level:   # meaning need to add a new [] to save that level
            result.append([node.val])
        else:
            result[level].append(node.val)
        helper(node.left, level+1)
        helper(node.right,level+1)
    result = []
    helper(root, 0)
    return result

In [10]:
from collections import deque
def levelOrder_2(root):
    """BFS O(N)"""
    queue = deque([(root, 0)])
    result = []
    while queue:
        node, level = queue.popleft()
        if len(result) == level:   # a new level should be added to the result
            result.append([])  # create a space in result to save that new level
        result[level].append(node.val)
        if node.left:
            queue.append((node.left, level+1))
        if node.right:
            queue.append((node.right, level+1))
    return result
            
    

In [11]:
def levelOrder_3(root):
    """BFS, but dont pop item from queue"""
    if not root:
        return []
    result = []
    queue = [root]
    while queue:
        next_queue = []
        result.append([])  # create a new space to save item from queue => current queue
        for node in queue:  # loop over current queue, which has nodes of the same levle
            result[-1].append(node.val)  # -1 locates to the last[] added to the result
            if node.left:
                next_queue.append(node.left)
            if node.right:
                next_queue.append(node.right)
        queue= next_queue   # update the queue, to go to next level of nodes
    return result
        
             
        

another breath first search but no need to pass a next_queue var

In [5]:
from collections import deque
def levelOrder(root):
    """dFS"""

    if not root:
        return []
    result = []
    queue = deque([root])
    while queue:
        size = len(queue) # using this size to determine the next level nodes to pop out
        result.append([])
        for _ in range(size):
            node = queue.popleft()
            result[-1].append(node.val)
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
    return result

<b>Binary Tree Level Order Traversal II </b>   
way 1: this question can use the same methods above, but return the reversed result list  
way 2: using special data structure to avoid reverse

In [1]:
from collections import deque
def levelOrderBottom(root):
    if not root:
        return []
    queue = [root]
    result = deque()
    while queue:
        next_queue = []
        result.appendleft([])  # add a new [] but to the left => so make latter level comes first
        for node in queue:
            result[0].append(node.val)   # here result[0] compare to result[-1]
            if node.left:
                next_queue.append(node.left)
            if node.right:
                next_queue.append(node.right)
        queue = next_queue
    return result
            

<b>103. binary tree zigzag level order traversal </b> 

very similar to question <b>102</b>  
here the way 1 corresponding to 102's way 3:

visit the nodes level by level, if the level is an odd level, revese the value  
how to update the next queue is the same as 102, breath first search. but how to save the value into result is where the zigzag happenning

In [ ]:
class Solution:
    def zigzagLevelOrder(root):
        if not root:
            return []
        result = []
        queue = [(root, 0)]
        while queue:
            next_queue = []
            for node, level in queue:
                if len(result)==level:
                    result.append([])
                result[level].append(node.val)
                if node.left:
                    next_queue.append((node.left, level+1))
                if node.right:
                    next_queue.append((node.right, level+1))
            if level%2==1:
                # from right to left
                result[level] = result[level][::-1]
            queue = next_queue
 
        return result

way 2 corresponding to <b>102</b>'s way 1, DFS way  

how to loop through each node is the same: DFS, preorder, node => left => right.   
how to save the result into the result list is different depending on what level the node is in. => where the deque.appendleft() is used


In [23]:
def zigzagLevelOrder(root):
    def helper(node, level):
        if not node:
            return
        if len(result) == level:
            result.append(deque([node.val]))
        else:
            if level%2==0:
                # result is from left to right
                result[level].append(node.val)
            else:
                # result should be from right to left
                result[level].appendleft(node.val)
        # the preorder traversal of DFS is not changed
        helper(node.left, level+1)
        helper(node.right, level+1)

    result = []  # a list of deque
    helper(root, 0)
    return result
        

<b> 733 Flood hill </b>  

similar as the question: number of islands, but simpler. no need to loop the entire array, only need to start at the [sr][sc] point

In [11]:
def floodFill(image, sr, sc, newColor):
    """similar as number of islands question"""

    def helper(i, j):
        # index out of bound or 
        if i<0 or j<0 or i>=n or j>=m or image[i][j]!=color:
            return None
        image[i][j] = newColor
        helper(i+1, j)
        helper(i-1, j)
        helper(i, j+1)
        helper(i, j-1)

    n = len(image)
    if n == 0: return image
    m = len(image[0])
    if m == 0: return image
    color = image[sr][sc]
    if newColor == color: return image
    # only need to check from the point [sr][sc], no need to loop entire array
    helper(sr, sc)
    return image


In [12]:
image = [[1,1,1],[1,1,0],[1,0,1]]
sr = 1
sc = 1 
newColor = 2
floodFill(image, sr, sc, newColor)

[[2, 2, 2], [2, 2, 0], [2, 0, 1]]

<b> 98 validate binary search tree</b>  

<b>way one </b>  => remember this one  
using float(-inf) and float(inf) to initiate the range of value. if node's value is not within its corresponding range, then it is not a BST

In [13]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
def isValidBST(root):
    
    def helper(node, min_val, max_val):
        # node.val should be within the range [min_val, max_val]
        if not node:
            return True
        if node.val >= max_val or node.val <=min_val:
            return False
        return helper(node.left, min_val, node.val) and helper(node.right, node.val, max_val)
    
    return helper(root, float('-inf'), float('inf'))

In [15]:
root = TreeNode(2)
root.left = TreeNode(1)
root.right= TreeNode(3)
isValidBST(root)

True

<b> way two </b>  
same as above but using stack and while loop

In [17]:
def isValidBST(root):
    """
    if it is binary search tree, should 
    get left < root < right
    """

    if not root:
        return True
    stack = [(root, float('-inf'), float('inf'))]
    while stack:
        node, lower, higher = stack.pop()
        if not node:
            continue
        val = node.val
        # value not within the range to be a valid BST
        if val>=higher or val<=lower:
            return False
        else:
            stack.append((node.left, lower, val))
            stack.append((node.right, val, higher))
    return True

<b>way three</b>.  
inorder traversal, a bit harder to understand comparing with the way above

the inorder var is the left boundry value for each node and it keeps updating

In [20]:
def isValidBST(root):
    stack, inorder = [], float('-inf')
    while stack or root:
        # go from top to bottom left
        while root:
            stack.append(root)
            root=root.left
        # first node is the very left bottom node, from there to the top
        root = stack.pop()
        if root.val <= inorder:
            return False
        inorder = root.val # new left boundary
        root=root.right
    return True

In [21]:
root = TreeNode(2)
root.left = TreeNode(1)
root.right= TreeNode(3)
isValidBST(root)

True

<b>101. Symmetric tree </b>

recursive call! have a helper function compare left and right.   
symmetric, so left's left vs right's right, left's right vs right's left

In [22]:
def isSymmetric(root):
    """left should == right"""
    def helper(left, right):
        # compare two nodes
        # all empty
        if (not left) and (not right):
            return True
        # all non-empty
        if left and right:
            # NOTE: symmetric, so left's left vs right's right, left's right vs right's left
            return (left.val==right.val) and helper(left.left, right.right) and helper(left.right, right.left)
        # one empty one is not
        return False

    if not root:
        return True
    return helper(root.left, root.right)

<b> 222 count complete tree nodes </b>

complete tree: every level, except the last, is guaranteed to be completely filled. all nodes in the last level are as far left as possible

<b> way 1 </b>  
recursive. just loop through each node and count. O(N)

In [1]:
def countNodes(root):
    """loop over each node"""
    return 1 + self.countNodes(root.left)+self.countNodes(root.right) if root else 0

<b> way 2</b>  
iteration way to loop over all nodes. O(N)

In [2]:
def countNodes(root):
    """level by level"""
    if not root:
        return 0
    queue = deque([root])
    result = 0
    while queue:
        node = queue.popleft()
        result += 1
        if node.left:
            queue.append(node.left)
        if node.right:
            queue.append(node.right)
    return result

<b> way 3 </b>  
fully leverage the "complete tree" attritubes!  
for a complete tree with totally d level, the number of nodes excluding the last one is   
2^0 + 2^1+...+2^(d-1) = 2^(d)-1
the number in the last level can be varied from 1 to 2^d
so first get to know how many levels the tree has, then find how many nodes there are in the last level.

using Binary Search!


In [ ]:
def countNodes(root):
    """level by level, leverage the complete tree attributes"""
    def compute_depth(node):
        """return tree depth in O(d) time
            using the left most path is enough to get the depth
            here level start from 0, NOT 1, represent depth!
        """
        d = 0
        while node.left:
            node = node.left
            d += 1
        return d

    def exists(ind, depth, node):
        """
        number of nodes in last level is no more than 2**depth, so the index range is from 0 to 2**depth -1 
        """
        # define the binary search range
        left, right = 0, 2**depth-1
        for _ in range(depth):
            pivot = left+(right-left)//2
            if ind <= pivot:
                node = node.left
                right = pivot
            else:
                node = node.right
                left = pivot + 1
        # after going down to the last depth, the node is the position where the index should locate to, if the node is none, then the index does not exist
        return node is not None

    if not root:
        return 0
    depth = compute_depth(root)  # O(d)
    if depth == 0:
        return 1
    # perform binary search to see how many nodes are there in the last level
    # find the last node in the last level
    left, right = 0, 2**depth-1 # 0 index based
    while left <= right:
        pivot = left+(right-left)//2
        if exists(pivot, depth, root):
            left = pivot+1
        else:
            right = pivot-1

    # nodes number in all previous levels are 2**d-1
    # when out of the while loop, left is off the last node by 1 position, so happen to work out here even though range is starting from 0
    return (2**depth-1) + left        

another easier understandable way => <b> a good one </b>.    
compare the depth between left sub stree and right sub tree.  
if it is equal, left sub tree is a perfect binary tree (fill with full), look into the sub tree of the right part.
if it is not equal, stop point found, right sub stree's last level is the stop point.  
time complexity O(logN * logN): LogN to find depth, need to run coutNodes for logN time, so LogN*logN. Or d for depth, get depth takes O(d), there is one finding depth action for every d, so O(d*d)

In [8]:
class Solution:
    def get_depth(self, node):
        """O(logN) every time"""
        # only go through the left tree is enough
        return 1+self.get_depth(node.left) if node else 0
    def countNodes(self, root):
        """O(logN) every time"""
        if not root:
            return 0
        left_depth = self.get_depth(root.left)
        right_depth = self.get_depth(root.right)
        if left_depth == right_depth:
            # left tree is full, look at the right tree
            # 1: the count for the root
            # (2**left_depth-1): the node number for the full left tree
            return 1 + (2**left_depth-1) + self.countNodes(root.right)
        else:
            # in the last level, the right tree is empty, the node only exist in the left sub tree
            return 1 + (2**right_depth-1) + self.countNodes(root.left)

<b> 951 flip equivalent binary tree </b>  
YEAH I DID IT!  
the question just require some number of flip operations, but all flipped, which is mirror tree. preorder traversal. When do comparision, the either they should be identical, or flipped on the left and right value, and the root value should be same

time compleixty:  O(N) or O(min(N1, N2) to be more specific
because the constraints of value are distinct, at most 2 out of the 4 recursive calls could go all the way down to the bottom

In [3]:
def flipEquiv(root1, root2):
    """bottom up DFS => post order"""

    def dfs_helper(node1, node2):
        if not node1 and not node2:
            return True
        elif node1 and node2:
            # root value should be the same
            criteria_1 = node1.val == node2.val
            # 2 and 3 are for comparing left and right value, shoud be either identical or flipped
            # identical but not mirror
            criteria_2 = (dfs_helper(node1.left, node2.left) and dfs_helper(node1.right, node2.right))
            # mirror/ flipped
            criteria_3 = (dfs_helper(node1.left, node2.right) and dfs_helper(node1.right, node2.left))
            return criteria_1 and (criteria_2 or criteria_3)
            # return (node1.val == node2.val) and ( (dfs_helper(node1.left, node2.left) and dfs_helper(node1.right, node2.right)) or (dfs_helper(node1.left, node2.right) and dfs_helper(node1.right, node2.left)))

        else:
            return False

    if not root1 and not root2:
        return True
    return dfs_helper(root1, root2)

<b> clean up a bit from above </b>

In [5]:

def flipEquiv(root1, root2):
    """bottom up DFS => post order"""

    if not root1 or not root2:
        return root1==root2==None
    identical = (self.flipEquiv(root1.left, root2.left) and self.flipEquiv(root1.right, root2.right))
    mirror =  (self.flipEquiv(root1.left, root2.right) and self.flipEquiv(root1.right, root2.left))
    return root1.val == root2.val and (identical or mirror)



<b> 99 Recover Binary Search Tree </b>

a very inspiring solution. sometimes simple algo works perfectly, not even need fancy one.  
binary search tree: in in-order traversal you should get a sorted array, but since two nodes are mis-ordered, there will be two pair of nodes who do not satisfy pre_node val < cur_node val. Find the two node, and then swap their value.

remember: TreeNode is a complex class, so var is a reference to it, which makes swapping value possible

the original explanation post is [here](https://leetcode.com/problems/recover-binary-search-tree/discuss/32535/No-Fancy-Algorithm-just-Simple-and-Powerful-In-Order-Traversal)

for example: 6,3,4,5,2 => 6,3 and 5,2 not satisfy the binary search tree definition. 6 is the first node, 2 is the second node, swap the value of the two, done

In [ ]:
class Solution:
    def __init__(self):
        self.pre_node = TreeNode(float('-inf'))
        self.first_node = None 
        self.second_node = None
    def find_nodes(self, node):
        """find the two nodes that should swap back
        in-order traversal, 
            first node is the one larger than parent node
            second node is the one smaller than parent node
        """
        if not node:
            return
        self.find_nodes(node.left)
        
        # find the two mistaken nodes
        if not self.first_node and self.pre_node.val > node.val:
            # find the first node
            self.first_node = self.pre_node
        # only assign second item when first item is found
        # NOTE this not work: if not self.second_item and self.pre_node.val > node.val => because there will be two pair of unqualified nodes, we want the second pair
        if self.first_node and self.pre_node.val > node.val:
            # find the second node
            self.second_node = node
        self.pre_node = node
        
        self.find_nodes(node.right)
        
    def recoverTree(self, root: TreeNode) -> None:
        """
        Do not return anything, modify root in-place instead.
        """
        self.find_nodes(root)
        self.first_node.val, self.second_node.val = self.second_node.val, self.first_node.val
        

<b> 105 Construct binary tree from preorder and inorder traversal </b>

Recursion, using preorder to find next root, using inorder to separate the left and right subtree.   
<b>Way 1</b> starting point:   
the most intuitive one but not optimized. Pop(0) is O(N), list.index(val) is also O(N). Then need to do N time for the recursion call to construct each node. So O(N^2). Also passing the copy of sliced inorder list takes extra space.  

In [1]:
def buildTree(preorder, inorder):
    """recursion, using preorder list to find root node, using indorder list to separate left tree and right tree"""
    if inorder:
        root_ind = inorder.index(preorder.pop(0))
        root = TreeNode(inorder[root_ind])
        root.left = self.buildTree(preorder, inorder[0:root_ind])
        root.right = self.buildTree(preorder, inorder[root_ind+1:])
        return root

<b>Way 2</b>: some optimization. Quicker than way 1

Instead of pop(0), can reverse the preorder list to use pop() which is only O(1). Instead of passing copy of sliced inorder list, can pass the index range. But list.index(val) is still used, which is O(N), also N recursion call. Still O(N^2), but space usage is much smaller

preorder is shortening, while inorder list is intact

In [2]:
def buildTree(preorder, inorder):
    """recursion, using preorder list to find root node, using indorder list to separate left tree and right tree"""
    def helper(l, r):
        # l and r are valid index, all included
        if l > r:
            return None

        root_ind = inorder.index(preorder.pop())
        root = TreeNode(inorder[root_ind])
        root.left = helper(l, root_ind-1)
        root.right = helper(root_ind+1, r)
        return root

    preorder.reverse()
    l = len(inorder)
    return helper(0, l-1)

<b> Way 3</b>: further optimization to O(N)

constructing a dictionary to avoid using list.index(val) method to locate root index and separate the left and right subtree, overall time is down to O(N) but have some extra space.


In [3]:
def buildTree(preorder, inorder):
    """recursion, using preorder list to find root node, using indorder list to separate left tree and right tree"""
    def helper(l, r):
        # l and r are valid index, all included
        if l > r:
            return None

        root_ind = ind_map[next(preorder)]
        root = TreeNode(inorder[root_ind])
        root.left = helper(l, root_ind-1)
        root.right = helper(root_ind+1, r)
        return root

    preorder = iter(preorder)
    ind_map = {}
    for i, num in enumerate(inorder):
        ind_map[num] = i
    l = len(inorder)
    return helper(0, l-1)

<b> 106 construct binary tree from inorder and postorder traversal </b>

same as above, but using postorder to find the root, then construct the tree in reversd postorder order, which is root, right, left. The item popping out of the postorder list is the root for the reversed order

In [4]:
def buildTree(inorder, postorder):
    """recursive, postorder to find the root, inorder to separate the left and right subtree"""

    def helper(l, r):
        if l > r:
            return None
        # l, r are valid index for item in inorder list
        root_ind = ind_map[postorder.pop()]
        # reverse the postorder order to construct the tree
        root = TreeNode(inorder[root_ind])
        root.right = helper(root_ind+1, r)
        root.left = helper(l, root_ind-1)
        return root

    ind_map = {}
    for i, num in enumerate(inorder):
        ind_map[num] = i
    l=len(inorder)
    return helper(0, l-1)

further possible optimization: not using pop to modify the postorder list, but using index or other ways. It seems not good to touch/modify input value

In [5]:
def buildTree(inorder, postorder):
    """recursive, postorder to find the root, inorder to separate the left and right subtree"""

    def helper(l, r):
        if l > r:
            return None
        # l, r are valid index for item in inorder list
        root_val = postorder[length-1 - next(post_ind)]
        root_ind = ind_map[root_val]
        # reverse the postorder order to construct the tree
        root = TreeNode(root_val)
        root.right = helper(root_ind+1, r)
        root.left = helper(l, root_ind-1)
        return root

    ind_map = {}
    for i, num in enumerate(inorder):
        ind_map[num] = i
    length=len(inorder)
    post_ind = iter(range(length))
    return helper(0, length-1)

<b> 257. Binary Tree Paths </b>

the part that stucks me is what is the base case. if return [] wont work because ['2' + i for i in []] will return []. The solution below separate into 2 base case, one is root node is none (only check for root node!) the other is leave node (so leave node's left and right wont be checked using the root node case).  
way 1: dfs way, base case is leave node and empty root node.

In [1]:
def binaryTreePaths(root):
        
    if not root: # this one only check root node, so only used once. leave node's left and right None wont reach here.
        return []
    if not root.left and not root.right:
        # reaching the leave => the [] is checked once, only for root, not for leave
        return [str(root.val)]

    return [str(root.val)+'->'+i for i in binaryTreePaths(root.left)]+[str(root.val)+'->'+i for i in binaryTreePaths(root.right)]


In [2]:
 ['2' + i for i in []]  # see it will return [] not ['2']

[]

way 2: using stack and dfs.  
why dfs: because stack.pop will return the most recently added node. so the first returned is the left most leave.

In [3]:
def binaryTreePaths(root):
        
    """dfs + stack"""
    if not root:
        return []
    res, stack = [], [(root, "")]
    while stack:
        node, s = stack.pop()
        if not node.left and not node.right:
            # a leave node
            res.append(s+str(node.val))
        if node.right:
            stack.append((node.right, s+str(node.val)+'->'))
        if node.left:
            stack.append((node.left, s+str(node.val)+'->'))
    return res

way 3: using deque and bfs  
why bfs: deque can popleft, which popping the node in the order added, => same as looping through node in each level from top to bottom.

In [1]:
from collections import deque

def binaryTreePaths(root):
    """bfs + stack"""
    if not root:
        return []
    res, stack = [], deque([(root, "")])
    while stack:
        node, s = stack.popleft()
        if not node.left and not node.right:
            # a leave node
            res.append(s+str(node.val))
        if node.right:
            stack.append((node.right, s+str(node.val)+'->'))
        if node.left:
            stack.append((node.left, s+str(node.val)+'->'))
    return res

<b> 112 Path Sum </b>

way 1: DFS + recursive, loop through all root to leaf path  
edge case: [] 0 => still False, also targetSum and node value can be negative

In [2]:
def hasPathSum(root, targetSum: int) -> bool:
    """DFS search, recursive too"""
    # base case
    if not root:
        return False
    if not root.left and not root.right and targetSum==root.val:
        return True

    return self.hasPathSum(root.left, targetSum-root.val) or self.hasPathSum(root.right, targetSum-root.val)

way 2: DFS + stack, left tree, then right tree. It can stop early when find a solution

In [3]:
def hasPathSum(root, targetSum: int) -> bool:
    """DFS search, using stack"""

    if not root:
        return False
    stack = [(root, root.val)]
    while stack:
        node, val = stack.pop()
        if not node.left and not node.right and val == targetSum:
            return True
        # since using pop, last one added will be used first
        # so left tree will be traversed first
        if node.right:
            stack.append((node.right, node.right.val+val))
        if node.left:
            stack.append((node.left, node.left.val+val))
    return False


<b> 113 Paths Sum II </b>

using the stack way as problem 112. ( the DFS recursive way works too, just with an extra helper function)

In [4]:
def pathSum(root, targetSum):
    if not root:
        return []
    res = []
    stack = [(root, root.val, [])]
    while stack:
        node, val, path = stack.pop()
        if not node.left and not node.right and val == targetSum:
            path.append(node.val)
            res.append(path)
        if node.right:
            stack.append((node.right, node.right.val+val, path+[node.val]))
        if node.left:
            stack.append((node.left, node.left.val+val, path+[node.val]))
    return res

can further reduce the space complexity by using backtracking. the solution above passing the [node.val] and create a new list everytime when adding a new node, it is space-consuming.

In [5]:
def pathSum(root, targetSum):
    if not root:
        return []
    res = []
    path = []
    def helper(node, targetSum, path, res):
        """backtracking"""
        if not node:
            return
        path.append(node.val) # start backtracking
        if not node.left and not node.right and node.val==targetSum:
            res.append(path[:]) # [] is passed in reference
        else:
            helper(node.left, targetSum - node.val, path, res)
            helper(node.right, targetSum - node.val, path, res)
        path.pop() # backtracking resume to previous status before recursion

    helper(root, targetSum, path, res)
    return res
